<a href="https://colab.research.google.com/github/sungjin417/sesac-phython/blob/master/%EA%B5%90)%EB%8C%80%EA%B7%9C%EB%AA%A8_LLM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%A7%80%EC%8B%9D_%EC%B1%97%EB%B4%87_%EA%B0%9C%EB%B0%9C_3%EC%9D%BC%EC%B0%A8(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🌼 대규모 LLM을 활용한 지식 챗봇 개발 - 3차시(24.11.25)

---

In [18]:
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2LMHeadModel
import pandas as pd
import tqdm

In [19]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.4.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
train_data = pd.read_excel('/content/drive/MyDrive/ChatbotData.xlsx')

In [22]:
print(f'데이터 개수 : {len(train_data)}')

데이터 개수 : 11823


In [23]:
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [24]:
del train_data['label']

In [25]:
train_data.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [26]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id] # 문장의 시작 토큰
    eos_token = [tokenizer.eos_token_id] # 문장의 끝  토큰
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)
    yield bos_token + sent + eos_token # return이랑 비슷하지만 필요할때만 리턴 따라서 데이터 사용에 용이함

In [27]:
batch_size = 16
dataset = tf.data.Dataset.from_generator(get_chat_data, output_signature=tf.TensorSpec(shape=(None,), dtype=tf.int32))

In [28]:
dataset = dataset.padded_batch(batch_size, padded_shapes = (None,), padding_values = tokenizer.pad_token_id)

In [29]:
for batch in dataset:
  print(batch)
  break

tf.Tensor(
[[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
   7098 25856     1     3     3     3     3     3     3]
 [    1     2  9020  8263  7497 10192 11615  8210  8006     4 12422  8711
   9535  7483 12521     1     3     3     3     3     3]
 [    1     2  9085  7597   395  8149 10624  7397 24224 13358  7182     4
  12079  8135 16899  9677  8234   389     1     3     3]
 [    1     2  9085  7597   395  8149  9465 10624  7397 24224 13358  7182
      4 12079  8135 16899  9677  8234   389     1     3]
 [    1     2  9943   422   418  9327  8702  7098     4  9847 16912 18328
   8671  7415  8263  8234   389     1     3     3     3]
 [    1     2  9815   410 21249 10174  6824  8210  8006     4  9427 11056
  11594 10137 10556  9266  8711 25856     1     3     3]
 [    1     2  9815   410 21249  9183  7249     4  9427 11056 11594 10137
  10556  9266  8711 25856     1     3     3     3     3]
 [    1     2  9815 37655  9622  8619 10401  9183  9328   216     4  944

In [30]:
print(tokenizer.decode(batch[0]))
# [    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028  7098 25856     1     3     3     3     3     3     3]

</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad>


In [31]:
adam = tf.keras.optimizers.Adam(learning_rate = 3e-5, epsilon = 1e-08)
# epsilon : 0으로 나눠지는 상황을 방지하기 위해 아주 작은 값을 추가

In [17]:
steps = len(train_data) // batch_size + 1

In [35]:
epochs = 3
for epoch in range(epochs):
  epoch_loss = 0
  for batch in tqdm.tqdm(dataset, total = steps):
    with tf.GradientTape() as tape: # 손실값의 가중치를 자동으로 계산
      result = model(batch, labels = batch)
      loss = result[0]
      batch_loss = tf.reduce_mean(loss)
      # loss값을 평균화 해서 batch_loss에 입력
    grads = tape.gradient(batch_loss, model.trainable_variables)
    # 학습 가능한 변수(trainle_variables) 의 기울기를 계산
    adam.apply_gradients(zip(grads, model.trainable_variables))
    epoch_loss += batch_loss/steps
  print(f'[epoch : {epoch + 1}] cost = {epoch_loss:>.9}')

100%|██████████| 739/739 [08:21<00:00,  1.47it/s]


[epoch : 1] cost = 2.23738623


100%|██████████| 739/739 [08:21<00:00,  1.47it/s]


[epoch : 2] cost = 1.75022101


100%|██████████| 739/739 [09:21<00:00,  1.32it/s]

[epoch : 3] cost = 1.36097479


In [50]:
text = '오늘도 좋은 하루!'
sent = '<usr>' + text + '<sys>'
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])

In [51]:
print(f'정수 인코딩 : {input_ids}')

정수 인코딩 : [[    1     2 10070  7235 10586 12557   376     4]]


In [52]:
print(f'정수 인코딩을 디코딩 : {tokenizer.decode(input_ids[0])}')

정수 인코딩을 디코딩 : </s><usr> 오늘도 좋은 하루!<sys>


In [54]:
output = model.generate(input_ids, max_length = 50, early_stopping=True, eos_token_id = tokenizer.eos_token_id)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [58]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence)

</s><usr> 오늘도 좋은 하루!<sys> 오늘도 좋은 하루로 기억될 거예요.</s>


In [62]:
print(decoded_sentence.split('<sys> ')[1].replace('</s>',''))

오늘도 좋은 하루로 기억될 거예요.


In [64]:
output = model.generate(input_ids, max_length = 100, do_sample = True, top_k = 10)
# do_sample = True : 텍스트 생성 시 확률 기반으로 랜덤성을 추가
# top_k = 10 : 확률이 높은 상위 10개
decode_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence.split('<sys> ')[1].replace('</s>', ''))

오늘도 좋은 하루로 기억될 거예요.


In [71]:
def return_answer(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length = 50, do_sample = True, top_k = 5)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_respone = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_respone

In [72]:
return_answer('안녕, 반가워')

'안녕.'

In [74]:
return_answer('너 딥러닝 잘해?')

'사람마다 다르겠지만 잘 해요.'

In [75]:
return_answer('오늘 날씨 어때?')

'오늘 날씨로 마음이 편한데요.'

In [76]:
return_answer('내일 날씨는 어때?')

'날씨 어플을 실행해보세요.'